In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/UKESM1-0-LL/NDJF/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values

ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        ...,
        [-9.20060351e-02, -8.75295624e-02, -8.35101083e-02, ...,
         -1.02113038e-01, -9.88488346e-02, -9.52089056e-02],
        [-2.37748101e-02, -2.22768579e-02, -2.07939986e-02, ...,
         -2.70576365e-02, -2.60799583e-02, -2.48867963e-02],
        [ 1.58317778e-02,  1.60615165e-02,  1.62498783e-02, ...,
          1.54889300e-02,  1.56318359e-02,  1.57701410e-02]],

       [[ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(19800,)


In [5]:
AO_ds.to_netcdf('UKESM1-0-LL-AOindex-NDJF-Daily-1980-2014.nc')

In [25]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.1763637, dtype=float32)
Coordinates:
    time     object 1875-02-30 12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [24]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.9811716, dtype=float32)
Coordinates:
    time     object 1954-01-30 12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [10]:
zLevData = xr.open_dataset('../../../Data/CMIP6/UKESM1-0-LL/NDJF/zgNH_NDJFday_UKESM1-0-LL_historical_r1i1p1f2_gn_18500101-20141230.nc')

In [11]:
z500=zLevData.zg.sel(plev=50000)

In [12]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [13]:
#z145E40N

In [14]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [15]:
#z20E55N

In [16]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [17]:
#z75E55N

In [18]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [19]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [20]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [21]:
EUVal

<xarray.DataArray 'zg' (time: 19800)>
array([ 0.48638194,  0.28331417,  0.08280063, ..., -0.36822364,
       -0.26021677, -0.16593594])
Coordinates:
  * time       (time) object 1850-01-01 12:00:00 ... 2014-12-30 12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 353 354 355 356 357 358 359 360
    lon        float64 145.3

In [22]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [23]:
EU_ds.to_netcdf('UKESM1-0-LL-EUindex-NDJF-Daily-1980-2014.nc')